SVM model on words, training to predict year.

In [9]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score #to asses the accuracy of the algorithm
import numpy as np
from sklearn.svm import LinearSVC #Linear Suppot Vector Classifier

In [10]:
df = pandas.read_csv("allData.csv", header = 0, na_values = '')

/home/zintegy/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
def getYear(date):
    year = int(date[2:4])
    if(80 <= year <= 84):
        return 0
    if(85 <= year <= 89):
        return 1
    if(90 <= year <= 94):
        return 2
    if(95 <= year <= 99):
        return 3
    if(0 <= year <= 4):
        return 4
    if(5 <= year <= 9):
        return 5
    if(10 <= year <= 14):
        return 6
    return -1


df['Year'] = df['Date'].map(getYear)
df = df[df['Year'] != -1]

In [12]:
df2 = df.sample(frac = 1,random_state = 0) #remove random_state

In [30]:
training = df2[:80000][['Headline', 'Year']].dropna()
test = df2[80000:100000][['Headline', 'Year']].dropna()

In [31]:
training

,Headline,Year
1198354,Sarah Eusden Is Engaged to Charles A. Gallop,2
1855815,"Paid Notice: Deaths ABRAM, MORRIS B.",4
1912635,"Under Circumstances, No Pomp as Clinton Signs ...",4
1172121,They Came to California for the Good Life; Now...,2
584575,NIKE INC reports earnings for Qtr to Aug 31,1
543095,CHERRY ELECTRICAL PRODCTS CORP reports earning...,1
3066646,Returning Upriver With Very Few Fish,6
3337175,Boys Don’t Run Away From These Princesses,6
2551345,Walk Tightropes. Teach Yoga. Fight Terrorists.,5
25175,Around the Nation; 29 Hurt and 300 Evacuated I...,0


In [32]:
#transform the 'body' column into a document term matrix
#tfidfvec = TfidfVectorizer(stop_words = 'english', min_df = 3, binary=True)
countvec = CountVectorizer(stop_words = 'english', min_df = 3, binary=True)

training_dtm_tf = countvec.fit_transform(training.Headline)
test_dtm_tf = countvec.transform(test.Headline)

#create an array for labels
training_labels = training.Year
test_labels = test.Year
test_labels.value_counts()

5    3304
6    3155
0    3064
1    2955
4    2681
3    2493
2    2348
Name: Year, dtype: int64

In [33]:
training_dtm_tf

<80000x14941 sparse matrix of type '<class 'numpy.int64'>'
	with 342259 stored elements in Compressed Sparse Row format>

In [34]:

svc = LinearSVC()
svc.fit(training_dtm_tf, training_labels)
predictions_svc = svc.predict(test_dtm_tf) 
accuracy_score(predictions_svc, test_labels)

0.3044